In [0]:
# Importando módulo de seguimiento 2025 sss
import sys
import importlib
sys.path.append('/Workspace/Users/luisfdiaz@bcp.com.pe/Modulo de Seguimiento/')

import SegScore as rmm
importlib.reload(rmm)

# Librerias y fuciones generales
from pyspark.sql.functions import date_format, expr, to_date, date_sub, add_months, col, when, coalesce, trim, broadcast, avg, max, min, lit, concat, window, round as colround, upper, abs as sparkabs,greatest
from pyspark.sql import functions as F
from pyspark import StorageLevel
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql.window import Window

In [0]:
bhv_hip_trad = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_base_bhv_hip_202504')

In [0]:
bhv_hip_trad = bhv_hip_trad.drop("TIPO_FONDEO", "CAMPANIA_AGRUPADA_CLI","MARCA_VULNERABLE_1_5")

In [0]:
bhv_hip_trad = bhv_hip_trad.withColumnRenamed("TIPO_FONDEO2", "TIPO_FONDEO")
bhv_hip_trad = bhv_hip_trad.withColumnRenamed("CAMPANIA_AGRUPADA_CLI2", "CAMPANIA_AGRUPADA_CLI")
bhv_hip_trad = bhv_hip_trad.withColumnRenamed("MARCA_VULNERABLE", "MARCA_VULNERABLE_1_5")

In [0]:
bhv_hip_trad = bhv_hip_trad.withColumn("TIPO_FONDEO", when(col("TIPO_FONDEO").isNull(), "NULO").otherwise(col("TIPO_FONDEO")))

In [0]:
bhv_hip_trad = bhv_hip_trad.filter(~col("TIPO_FONDEO").isin('GAHI', 'OTROS') & 
                                     ~col("TIPCREDITO").isin('HVIC', 'HVIM', 'HBIC', 'HSIC', 'HMIC', 'HMIM'))

In [0]:
bhv_hip_trad = bhv_hip_trad.withColumn("CODMES", col("CODMES").cast("integer"))
bhv_hip_trad = bhv_hip_trad.withColumn("DEF5", when(col("DISTANCIA_DEF") <= 5, lit(1)).otherwise(lit(0))) 
bhv_hip_trad = bhv_hip_trad.withColumn("DEF4", when(col("DISTANCIA_DEF") <= 4, lit(1)).otherwise(lit(0))) 

In [0]:
#forma 1: rellenamos los nulos con campaña agrupada ifrs9
bhv_hip_trad = bhv_hip_trad.withColumn("CAMPANIA_AGRUPADA_2_F", 
                   when(col("CAMPANIA_AGRUPADA_2") == "MISSING", col("CAMPANIA_AGRUPADA_IFRS9"))
                   .otherwise(col("CAMPANIA_AGRUPADA_2")))

In [0]:
bhv_hip_trad = bhv_hip_trad.withColumn("FLG_APALANCAMIENTO", 
                   when((col("CAMPANIA_AGRUPADA_CLI") == "TRADICIONAL") & (col("CUOTA_TOTAL_ING_U3M") > 0.70), "APALANCADO")
                   .when(col("CAMPANIA_AGRUPADA_CLI") == "TRADICIONAL", "NO APALANCADO")
                   .when((col("CAMPANIA_AGRUPADA_CLI") == "PYME") & (col("CUOTA_TOTAL_ING_U3M") > 1.50), "APALANCADO")
                   .when(col("CAMPANIA_AGRUPADA_CLI") == "PYME", "NO APALANCADO")
                   .otherwise("NO DEFINIDO"))

In [0]:
bhv_hip_trad = bhv_hip_trad.withColumn("FLG_APALANCAMIENTO2", 
                   when((col("CAMPANIA_AGRUPADA_CLI") == "TRADICIONAL") & (col("cuota_rcc_ing_u3m_t1") > 0.70), "APALANCADO")
                   .when(col("CAMPANIA_AGRUPADA_CLI") == "TRADICIONAL", "NO APALANCADO")
                   .when((col("CAMPANIA_AGRUPADA_CLI") == "PYME") & (col("cuota_rcc_ing_u3m_t1") > 1.50), "APALANCADO")
                   .when(col("CAMPANIA_AGRUPADA_CLI") == "PYME", "NO APALANCADO")
                   .otherwise("NO DEFINIDO"))

In [0]:
from pyspark.sql.functions import when, col, concat_ws

In [0]:
bhv_hip_trad = bhv_hip_trad.withColumn("ESTATUS_CEF", 
                   when(col("FLG_CEF") == "CON CEF", "CEF:1")
                   .otherwise("CEF:0"))

bhv_hip_trad = bhv_hip_trad.withColumn("ESTATUS_APAL_CEF", concat_ws("-", col("FLG_APALANCAMIENTO"), col("ESTATUS_CEF")))

bhv_hip_trad = bhv_hip_trad.withColumn("PERFIL_F1", 
                   when(col("FLG_APALANCAMIENTO") == "NO APALANCADO", "NO APALANCADO")
                   .when(col("ESTATUS_APAL_CEF") == "APALANCADO-CEF:1", "APALANCADO CON CEF")
                   .when(col("ESTATUS_APAL_CEF") == "APALANCADO-CEF:0", "APALANCADO EN RCC"))

bhv_hip_trad = bhv_hip_trad.withColumn("PERFIL_F1_VUL", 
                   when(col("FLG_APALANCAMIENTO") == "NO APALANCADO", "NO APALANCADO")
                   .otherwise(concat_ws("-", col("MARCA_VULNERABLE_1_5"), col("PERFIL_F1"))))

In [0]:
bhv_hip_trad = bhv_hip_trad.withColumn("PERFIL_F2", 
                   when(col("FLG_APALANCAMIENTO2") == "NO APALANCADO", "NO APALANCADO")
                   .when(col("ESTATUS_APAL_CEF") == "APALANCADO-CEF:1", "APALANCADO CON CEF")
                   .when(col("ESTATUS_APAL_CEF") == "APALANCADO-CEF:0", "APALANCADO EN RCC"))

bhv_hip_trad = bhv_hip_trad.withColumn("PERFIL_F2_VUL", 
                   when(col("FLG_APALANCAMIENTO2") == "NO APALANCADO", "NO APALANCADO")
                   .otherwise(concat_ws("-", col("MARCA_VULNERABLE_1_5"), col("PERFIL_F1"))))

In [0]:
bhv_hip_trad = bhv_hip_trad.withColumn("PD_BHV_CLI_HIP_6012_V2", expr("1 / (1 + exp(-XB_BHV_CLI_HIP_6012_V2))"))
bhv_hip_trad = bhv_hip_trad.withColumn("PD_CEF_FWL", expr("1 / (1 + exp(-XB_CEF_FWL))"))
bhv_hip_trad = bhv_hip_trad.withColumn("PD_BHV_HIP_CAL", expr("1 / (1 + exp(-XB_BHV_HIP_CAL))"))

In [0]:
bhv_hip_trad.persist(StorageLevel.MEMORY_AND_DISK).count()
bhv_hip_trad.groupBy('CODMES', 'CAMPANIA_AGRUPADA_CLI').agg(F.count("*")).show(50)

In [0]:
bhv_hip_trad = bhv_hip_trad.select('CODMES', 'CODCLAVECIC', 'CODCLAVECTA', 'CODCLAVEOPECTA', 'CAMPANIA_AGRUPADA_CLI', 'MTOPRINCIPAL_SOLES', 'pd_bhv_hip_25q1_1', 'PD_FABRICA', 'NUMEDADMADURACION_2', 'PERFIL_F2', 'PERFIL_F2_VUL', 'PERFIL_F1', 'FLG_APALANCAMIENTO', 'CODCOSECHA', 'CODCOSECHA_2', 'SEGMENTO_BANCA_F')

In [0]:
md_relcodclaveopectadl = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_MD_RELCODCLAVEOPECTADL')
md_relcodclaveopectadl = md_relcodclaveopectadl.withColumn("CODCLAVEOPECTA", col("codclaveopecta").cast("integer"))

# Tabla MIN_MES_DET: Obtener los minimos días de vencimiento 
hm_saldoprestamo = spark.table('catalog_lhcl_prod_bcp.bcp_udv_int_vu.h_saldocuentacreditopersonal')
hm_saldoprestamo = hm_saldoprestamo.withColumn("CODMES", date_format(col("FECDIA"), "yyyyMM"))
hm_saldoprestamo = hm_saldoprestamo.groupBy("CODMES", "CODCLAVECTA", "CTDDIAVCDA").agg(max("FECDIA").alias("MAX_FECDIA"))
min_mes_det = hm_saldoprestamo.groupBy("CODCLAVECTA").agg(
    min(when(col("CTDDIAVCDA") > 8, col("CODMES"))).alias("PRIMER_MES_8"),
    min(when(col("CTDDIAVCDA") > 15, col("CODMES"))).alias("PRIMER_MES_15"),
    min(when(col("CTDDIAVCDA") > 20, col("CODMES"))).alias("PRIMER_MES_20"),
    min(when(col("CTDDIAVCDA") > 30, col("CODMES"))).alias("PRIMER_MES_30")
)


# Cruzando para homologar el opecta
min_mes_det = min_mes_det.join(
    md_relcodclaveopectadl.selectExpr("CODCLAVECTADL as CODCLAVECTA", "CODCLAVEOPECTA"),
    on = ["CODCLAVECTA"],
    how = 'left'
)

min_mes_det.persist(StorageLevel.MEMORY_AND_DISK).count()

In [0]:
bhv_hip_trad = bhv_hip_trad.join(
    min_mes_det,
    on="codclaveopecta",
    how='left'
)

In [0]:
v_hm_univ_ctas_als = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.TABLASGE_v_hm_univ_ctas_als')
bhv_hip_trad = bhv_hip_trad.join(
    v_hm_univ_ctas_als.select("CODMES", "CODCLAVEOPECTA", "CTDDIAATRASO_F1", "CTDDIAATRASO_F2", "CTDDIAATRASO_F3"), 
    on=["CODCLAVEOPECTA", "CODMES"], 
    how='left'
)

In [0]:
# Tabla mm_mes_primer_default_norev: Obtener el minimo mes de default
mm_portafoliocredito = spark.table('catalog_lhcl_prod_bcp.bcp_ddv_adrmmgr_seginfobasesgenerales_vu.mm_portafoliocredito')

mm_mes_primer_default_norev = mm_portafoliocredito.join(
    md_relcodclaveopectadl.selectExpr("CODCLAVECTADL AS CODCLAVECTA", "CODCLAVEOPECTA"), 
    on = "CODCLAVECTA", 
    how="left"
)
bhv_hip_trad = bhv_hip_trad.drop('MIN_MES_DEF_60_BLOQ_REF_ACT')
bhv_hip_trad = bhv_hip_trad.join(
    mm_mes_primer_default_norev.select("codclaveopecta", coalesce(mm_mes_primer_default_norev["CODMESMINDEF60BLOQUEOREFIN"], lit(999999)).alias("MIN_MES_DEF_60_BLOQ_REF_ACT")),
    on="codclaveopecta",
    how='left'
)

In [0]:
base_seg = base_seg.withColumn(
    "PRIMER_MES_8", 
    when(col("PRIMER_MES_8").isNull(), 
    lit(999999)).otherwise(col("PRIMER_MES_8"))
)

base_seg = base_seg.withColumn(
    "DISTANCIA_8", 
    (F.round(col("PRIMER_MES_8")/100, 0)*12 + (col("PRIMER_MES_8")-F.round(col("PRIMER_MES_8")/100, 0)*100)) - 
    (F.round(col("CODMES")/100, 0)*12 + (col("CODMES")-F.round(col("CODMES")/100, 0)*100))
)

base_seg = base_seg.withColumn("DET_8_0", when(col("DISTANCIA_8") <= 0, lit(1)).otherwise(lit(0)))
base_seg = base_seg.withColumn("DET_8_2", when(col("DISTANCIA_8") <= 2, lit(1)).otherwise(lit(0)))
base_seg = base_seg.withColumn("DET_8_3", when(col("DISTANCIA_8") <= 3, lit(1)).otherwise(lit(0)))
base_seg = base_seg.withColumn("DET_8_4", when(col("DISTANCIA_8") <= 4, lit(1)).otherwise(lit(0)))

base_seg = base_seg.withColumn(
    "PRIMER_MES_15", 
    when(col("PRIMER_MES_15").isNull(), 
    lit(999999)).otherwise(col("PRIMER_MES_15"))
)

base_seg = base_seg.withColumn(
    "DISTANCIA_15", 
    (F.round(col("PRIMER_MES_15")/100, 0)*12 + (col("PRIMER_MES_15")-F.round(col("PRIMER_MES_15")/100, 0)*100)) - 
    (F.round(col("CODMES")/100, 0)*12 + (col("CODMES")-F.round(col("CODMES")/100, 0)*100))
)

base_seg = base_seg.withColumn("DET_15_0", when(col("DISTANCIA_15") <= 0, lit(1)).otherwise(lit(0)))
base_seg = base_seg.withColumn("DET_15_2", when(col("DISTANCIA_15") <= 2, lit(1)).otherwise(lit(0)))
base_seg = base_seg.withColumn("DET_15_3", when(col("DISTANCIA_15") <= 3, lit(1)).otherwise(lit(0)))
base_seg = base_seg.withColumn("DET_15_4", when(col("DISTANCIA_15") <= 4, lit(1)).otherwise(lit(0)))

base_seg = base_seg.withColumn(
    "PRIMER_MES_20", 
    when(col("PRIMER_MES_20").isNull(), 
    lit(999999)).otherwise(col("PRIMER_MES_20"))
)

base_seg = base_seg.withColumn(
    "DISTANCIA_20", 
    (F.round(col("PRIMER_MES_20")/100, 0)*12 + (col("PRIMER_MES_20")-F.round(col("PRIMER_MES_20")/100, 0)*100)) - 
    (F.round(col("CODMES")/100, 0)*12 + (col("CODMES")-F.round(col("CODMES")/100, 0)*100))
)


base_seg = base_seg.withColumn("DET_20_0", when(col("DISTANCIA_20") <= 0, lit(1)).otherwise(lit(0)))
base_seg = base_seg.withColumn("DET_20_2", when(col("DISTANCIA_20") <= 2, lit(1)).otherwise(lit(0)))
base_seg = base_seg.withColumn("DET_20_3", when(col("DISTANCIA_20") <= 3, lit(1)).otherwise(lit(0)))
base_seg = base_seg.withColumn("DET_20_4", when(col("DISTANCIA_20") <= 4, lit(1)).otherwise(lit(0)))

base_seg = base_seg.withColumn(
    "PRIMER_MES_30", 
    when(col("PRIMER_MES_30").isNull(), 
    lit(999999)).otherwise(col("PRIMER_MES_30"))
)

base_seg = base_seg.withColumn(
    "DISTANCIA_30", 
    (F.round(col("PRIMER_MES_30")/100, 0)*12 + (col("PRIMER_MES_30")-F.round(col("PRIMER_MES_30")/100, 0)*100)) - 
    (F.round(col("CODMES")/100, 0)*12 + (col("CODMES")-F.round(col("CODMES")/100, 0)*100))
)

base_seg = base_seg.withColumn("DET_30_0", when(col("DISTANCIA_30") <= 0, lit(1)).otherwise(lit(0)))
base_seg = base_seg.withColumn("DET_30_2", when(col("DISTANCIA_30") <= 2, lit(1)).otherwise(lit(0)))
base_seg = base_seg.withColumn("DET_30_3", when(col("DISTANCIA_30") <= 3, lit(1)).otherwise(lit(0)))
base_seg = base_seg.withColumn("DET_30_4", when(col("DISTANCIA_30") <= 4, lit(1)).otherwise(lit(0)))

base_seg = base_seg.withColumn('MORA_8_2',F.when(F.greatest(F.col('CTDDIAATRASO_F1'), F.col('CTDDIAATRASO_F2')) >= 8, 1).otherwise(0))
base_seg = base_seg.withColumn('MORA_15_2',F.when(F.greatest(F.col('CTDDIAATRASO_F1'), F.col('CTDDIAATRASO_F2')) >= 15, 1).otherwise(0))
base_seg = base_seg.withColumn('MORA_20_2',F.when(F.greatest(F.col('CTDDIAATRASO_F1'), F.col('CTDDIAATRASO_F2')) >= 20, 1).otherwise(0))
base_seg = base_seg.withColumn('MORA_30_2',F.when(F.greatest(F.col('CTDDIAATRASO_F1'), F.col('CTDDIAATRASO_F2')) >= 30, 1).otherwise(0))
base_seg = base_seg.withColumn('MORA_45_2',F.when(F.greatest(F.col('CTDDIAATRASO_F1'), F.col('CTDDIAATRASO_F2')) >= 45, 1).otherwise(0))

base_seg = base_seg.withColumn('MORA_1_3',F.when(F.greatest(F.col('CTDDIAATRASO_F1'), F.col('CTDDIAATRASO_F2'), F.col('CTDDIAATRASO_F3'))>= 1, 1).otherwise(0))
base_seg = base_seg.withColumn('MORA_8_3',F.when(F.greatest(F.col('CTDDIAATRASO_F1'), F.col('CTDDIAATRASO_F2'), F.col('CTDDIAATRASO_F3'))>= 8, 1).otherwise(0))
base_seg = base_seg.withColumn('MORA_15_3',F.when(F.greatest(F.col('CTDDIAATRASO_F1'), F.col('CTDDIAATRASO_F2'), F.col('CTDDIAATRASO_F3'))>= 15, 1).otherwise(0))
base_seg = base_seg.withColumn('MORA_20_3',F.when(F.greatest(F.col('CTDDIAATRASO_F1'), F.col('CTDDIAATRASO_F2'), F.col('CTDDIAATRASO_F3'))>= 20, 1).otherwise(0))
base_seg = base_seg.withColumn('MORA_30_3',F.when(F.greatest(F.col('CTDDIAATRASO_F1'), F.col('CTDDIAATRASO_F2'), F.col('CTDDIAATRASO_F3'))>= 30, 1).otherwise(0))
base_seg = base_seg.withColumn('MORA_45_3',F.when(F.greatest(F.col('CTDDIAATRASO_F1'), F.col('CTDDIAATRASO_F2'), F.col('CTDDIAATRASO_F3'))>= 45, 1).otherwise(0))

base_seg.persist(StorageLevel.MEMORY_AND_DISK).count()

base_seg = base_seg.withColumn(
    "MIN_MES_DEF", 
    when(col("MIN_MES_DEF_60_BLOQ_REF_ACT").isNull(), 
    lit(999999)).otherwise(col("MIN_MES_DEF_60_BLOQ_REF_ACT"))
)

base_seg = base_seg.withColumn(
    "DISTANCIA_DEF", 
    (F.round(col("MIN_MES_DEF")/100, 0)*12 + (col("MIN_MES_DEF")-F.round(col("MIN_MES_DEF")/100, 0)*100)) - 
    (F.round(col("CODMES")/100, 0)*12 + (col("CODMES")-F.round(col("CODMES")/100, 0)*100))
)

base_seg = base_seg.withColumn("DEF12", when(col("DISTANCIA_DEF") <= 12, lit(1)).otherwise(lit(0)))
base_seg = base_seg.withColumn("DEF11", when(col("DISTANCIA_DEF") <= 11, lit(1)).otherwise(lit(0)))
base_seg = base_seg.withColumn("DEF10", when(col("DISTANCIA_DEF") <= 10, lit(1)).otherwise(lit(0)))
base_seg = base_seg.withColumn("DEF9", when(col("DISTANCIA_DEF") <= 9, lit(1)).otherwise(lit(0)))
base_seg = base_seg.withColumn("DEF8", when(col("DISTANCIA_DEF") <= 8, lit(1)).otherwise(lit(0)))
base_seg = base_seg.withColumn("DEF7", when(col("DISTANCIA_DEF") <= 7, lit(1)).otherwise(lit(0)))
base_seg = base_seg.withColumn("DEF6", when(col("DISTANCIA_DEF") <= 6, lit(1)).otherwise(lit(0)))
base_seg = base_seg.withColumn("DEF5", when(col("DISTANCIA_DEF") <= 5, lit(1)).otherwise(lit(0)))
base_seg = base_seg.withColumn("DEF4", when(col("DISTANCIA_DEF") <= 4, lit(1)).otherwise(lit(0)))
base_seg = base_seg.withColumn("DEF3", when(col("DISTANCIA_DEF") <= 3, lit(1)).otherwise(lit(0)))

# Def foto
base_seg = base_seg.withColumn("DEF_60_BLOQ_REF_0", when(col("DISTANCIA_DEF") <= 0, lit(1)).otherwise(lit(0)))

base_seg.persist(StorageLevel.MEMORY_AND_DISK).count()

In [0]:
bhv_hip_trad = bhv_hip_trad.drop("codclavecta")
bhv_hip_trad.createOrReplaceTempView('temp_view')
spark.sql("CREATE OR REPLACE TABLE catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_base_bhv_hip_202505 LOCATION 'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/HIPOTECARIO_2025/BHV/202505/base_bhv_hip' AS SELECT * FROM temp_view")

In [0]:
hip_f = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_base_bhv_hip_202505')

In [0]:
pd_vig='PD_FABRICA' #PD_CEF_FWL PD_BHV_FWL
pd_cal='pd_bhv_hip_25q1_1'# en uso 
pd_trc='XB_BHV_CLI_HIP_6012_V2'
tipo_banda = 'Vasiseck'#Jeffrey Vasiseck
codmes_default = 202405
columna_monto='MTOPRINCIPAL_SOLES'
titulo='Calibración Behavior Hipotecario'

In [0]:
# Objeto de Seguimiento
Obj_Seg = rmm.MonitorScore_v01(
  nombre_modelo         = 'Personas Modelo Behavior',
  codigo_modelo         = 'MOD-BCP-20658',
  tier                  = 'II',
  detalle_seguimiento   = 'Seguimiento Behavior',
  mes_seguimiento       = '202502',
  base_monitoreo        = hip_f,
  pd1                   = pd_cal,
  pd2                   = pd_vig,
  monto_credito         = columna_monto,
  query_univ            = "CODMES >= 202201 AND DEF_60_BLOQ_REF_0=0 AND NUMEDADMADURACION_2>0",
  bandas                = tipo_banda,
  codmes_default        = codmes_default,
  meses_ventana_target  = 12,
  meses_atraso_target   = 2
)

In [0]:
hm_bhv_cli_cal, rc, rm, fc_1_trad, fm_1_trad = Obj_Seg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "CODMES >= 202205",
  pd_seg             = pd_vig,
  pd_comparacion     = False,
  proys_def_temp     = 6,
  proys_mor_temp     = 3,
  rd_aux             = 'RD6',
  rd_aux2            = 'MORA_30_3',
  mora_temp          = 'MORA_30_3',

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = 'Best_proy',
  suav_def_temp_xc   = 1,
  proy_mor_temp_xc   = 'Best_proy',
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14,

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = 'Best_proy',
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'Best_proy',
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,

  # Detalles del gráfico de calibración
  titulo             = 'Calibración Behavior Hipotercario Ahorro Local',
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.08,
  eje_y_xm           = 0.08,
  dim_grafico        = (22.5, 6),
  punt_mora          = 85,
  etiquetas          = True,
  tamaño_etiqueta    = 14,

  # Exportar factores para vista driver
  exportar_factores  = True
)

In [0]:
rm

In [0]:
rc

In [0]:
Obj_SegDis, evoldis ,evolks= Obj_Seg.EvolDis(
  query_filtro = "",
  pd_dis       = pd_vig,
  codmes_ini   = 202301,
  codmes_fin   = 202412,
  amplitud     = 1,
  disc_target= [12, 9,6],
  umbral_sup   = 0.5,
  umbral_inf   = 0.4,
  moviles      = False,
  etiquetas= True,
  tamaño_etiqueta=16,
  dim_grafico  = (22, 6),
  pos_leyenda  = (0.5, -0.3)
)

In [0]:
# Objeto de Seguimiento
Obj_Seg = rmm.MonitorScore_v01(
  nombre_modelo         = 'Personas Modelo Behavior',
  codigo_modelo         = 'MOD-BCP-20658',
  tier                  = 'II',
  detalle_seguimiento   = 'Seguimiento Behavior',
  mes_seguimiento       = '202502',
  base_monitoreo        = hip_f,
  pd1                   = 'CODMES',
  pd2                   = 'CODMES',
  monto_credito         = columna_monto,
  query_univ            = "CODMES >= 202201 AND DEF_60_BLOQ_REF_0=0 AND NUMEDADMADURACION_2>0",
  bandas                = tipo_banda,
  codmes_default        = codmes_default,
  meses_ventana_target  = 12,
  meses_atraso_target   = 2
)

In [0]:
hm_bhv_cli_cal, rc, rm, fc_1_trad, fm_1_trad = Obj_Seg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "CODMES >= 202205 AND CAMPANIA_AGRUPADA_CLI='TRADICIONAL'",
  pd_seg             = pd_vig,
  pd_comparacion     = False,
  proys_def_temp     = 6,
  proys_mor_temp     = 3,
  rd_aux             = 'RD6',
  rd_aux2            = 'MORA_30_3',
  mora_temp          = 'MORA_30_3',

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = 'All_proy',
  suav_def_temp_xc   = 1,
  proy_mor_temp_xc   = 'All_proy',
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14,

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = 'met3_xm',
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'All_proy',
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,

  # Detalles del gráfico de calibración
  titulo             = 'Calibración Behavior Hipotercario Tradicional',
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.05,
  eje_y_xm           = 0.05,
  dim_grafico        = (22.5, 6),
  punt_mora          = 85,
  etiquetas          = True,
  tamaño_etiqueta    = 14,

  # Exportar factores para vista driver
  exportar_factores  = True
)

In [0]:
rm

In [0]:
# from pyspark.sql.functions import col, concat, lit, to_date, add_months, date_format, when

# from pyspark.sql.functions import col, concat, lit, to_date, add_months, date_format, when

# Paso 1: Crear columna CODMES_COSECHA
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "CODMES_COSECHA", 
    F.date_format(
        add_months(
            to_date(F.concat(col("CODMES").cast("int"), lit("01")), "yyyyMMdd"),
            -col("NUMEDADMADURACION_2").cast("int")
        ),
        "yyyyMM"
    ).cast("int")
)

# Paso 2: Clasificación
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "COSECHITAS",
    when(col("CODMES_COSECHA") < 202204, "1. Anterior")
    .when((col("CODMES_COSECHA") >= 202204) & (col("CODMES_COSECHA") <= 202304), "2. 202204-202304")
    .when(col("CODMES_COSECHA") > 202304, "3. Posterior")
)

from pyspark.sql.functions import concat_ws, substring, when

# Extraer año y mes de CODMES_COSECHA
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "ANIO_COSECHA", substring(col("CODMES_COSECHA").cast("string"), 1, 4)
).withColumn(
    "MES_COSECHA", substring(col("CODMES_COSECHA").cast("string"), 5, 2).cast("int")
)

# Determinar el trimestre (Q1 o Q2)
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "TRIMESTRE_COSECHA",
    when(col("MES_COSECHA").between(1, 6), "Q1")
    .when(col("MES_COSECHA").between(7, 12), "Q2")
)

# Concatenar año y trimestre
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "SEMESTRE_COSECHA",
    concat_ws("-", col("ANIO_COSECHA"), col("TRIMESTRE_COSECHA"))
)

In [0]:
Obj_Seg.base_monitoreo.select("codmes", "CODMES_COSECHA", "NUMEDADMADURACION_2", "SEMESTRE_COSECHA").display()

In [0]:
driver='COSECHITAS'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_Seg.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = "CODMES >= 202209 AND CAMPANIA_AGRUPADA_CLI='PYME'",
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202201,
  codmes_fin1        = 202212,

  # Periodo reciente
  codmes_inicio2     = 202301,
  codmes_fin2        = 202412,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (22.5, 6),
  pos_leyenda        = (0.5, -0.25),
  etiquetas = True,
  borde=0.08
)

In [0]:
def print_res(sparkf_df):
  import pandas as pd
  pd.set_option('display.max_rows', None)
  pd.set_option('display.max_columns', None)
  pd_spark_df = sparkf_df.toPandas()
  
  return pd_spark_df

cubo = Obj_Seg.base_monitoreo.groupBy('CODMES', 'CAMPANIA_AGRUPADA_CLI', 'SEGMENTO_BANCA_F', 'COSECHITAS', 'SEMESTRE_COSECHA').agg(
    F.sum('MTOPRINCIPAL_SOLES').alias('M'),
    F.count("*").alias('M'),
    F.sum('DEF12')*F.sum(col("MTOPRINCIPAL_SOLES")).alias('DEF12'),
    F.sum('DEF6')*F.sum(col("MTOPRINCIPAL_SOLES")).alias('DEF6')
)

print_res(cubo)

In [0]:
hm_bhv_cli_cal, rc, rm, fc_1_trad, fm_1_trad = Obj_Seg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "CODMES >= 202205 AND CAMPANIA_AGRUPADA_CLI = 'TRADICIONAL' AND PERFIL_F1 = 'APALANCADO CON CEF'",
  pd_seg             = pd_vig,
  pd_comparacion     = False,
  proys_def_temp     = 6,
  proys_mor_temp     = 3,
  rd_aux             = 'RD6',
  rd_aux2            = 'MORA_30_3',
  mora_temp          = 'MORA_30_3',

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = 'Best_proy',
  suav_def_temp_xc   = 1,
  proy_mor_temp_xc   = 'det2_xc*0.7',
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14,

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = 'met3_xm',
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det3_xm',
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,

  # Detalles del gráfico de calibración
  titulo             = 'Calibración Behavior Hipotercario Tradicional',
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.12,
  eje_y_xm           = 0.12,
  dim_grafico        = (22.5, 6),
  punt_mora          = 85,
  etiquetas          = True,
  tamaño_etiqueta    = 14,

  # Exportar factores para vista driver
  exportar_factores  = True
)

In [0]:
hm_bhv_cli_cal, rc, rm, fc_2_trad, fm_2_trad = Obj_Seg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "CODMES >= 202205 AND CAMPANIA_AGRUPADA_CLI = 'TRADICIONAL' AND PERFIL_F1 = 'APALANCADO EN RCC'",
  pd_seg             = pd_vig,
  pd_comparacion     = False,
  proys_def_temp     = 6,
  proys_mor_temp     = 3,
  rd_aux             = 'RD6',
  rd_aux2            = 'MORA_30_3',
  mora_temp          = 'MORA_30_3',

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = 'Best_proy',
  suav_def_temp_xc   = 1,
  proy_mor_temp_xc   = 'Best_proy',
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14,

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = 'met3_xm',
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = "np.where(calxmontos['CODMES']==202502, det3_xm*1.05, det3_xm)",
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,

  # Detalles del gráfico de calibración
  titulo             = 'Calibración Behavior Hipotercario Tradicional',
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.12,
  eje_y_xm           = 0.12,
  dim_grafico        = (22.5, 6),
  punt_mora          = 85,
  etiquetas          = True,
  tamaño_etiqueta    = 14,

  # Exportar factores para vista driver
  exportar_factores  = True
)

In [0]:
hm_bhv_cli_cal, rc, rm, fc_3_trad, fm_3_trad = Obj_Seg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "CODMES >= 202205 AND CAMPANIA_AGRUPADA_CLI = 'TRADICIONAL' AND PERFIL_F1 = 'NO APALANCADO'",
  pd_seg             = pd_vig,
  pd_comparacion     = False,
  proys_def_temp     = 6,
  proys_mor_temp     = 3,
  rd_aux             = 'RD6',
  rd_aux2            = 'MORA_30_3',
  mora_temp          = 'MORA_30_3',

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = 'Best_proy',
  suav_def_temp_xc   = 1,
  proy_mor_temp_xc   = 'Best_proy',
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14,

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = 'met3_xm*0.95',
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det3_xm*1.1',
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,

  # Detalles del gráfico de calibración
  titulo             = 'Calibración Behavior Hipotercario Tradicional',
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.05,
  eje_y_xm           = 0.05,
  dim_grafico        = (22.5, 6),
  punt_mora          = 85,
  etiquetas          = True,
  tamaño_etiqueta    = 14,

  # Exportar factores para vista driver
  exportar_factores  = True
)

In [0]:
hm_bhv_cli_cal, rc, rm, fc_4_trad, fm_4_trad = Obj_Seg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "CODMES >= 202205 AND CAMPANIA_AGRUPADA_CLI = 'TRADICIONAL' AND PERFIL_F1 <> 'NO APALANCADO'",
  pd_seg             = pd_vig,
  pd_comparacion     = False,
  proys_def_temp     = 6,
  proys_mor_temp     = 3,
  rd_aux             = 'RD6',
  rd_aux2            = 'MORA_30_3',
  mora_temp          = 'MORA_30_3',

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = 'Best_proy',
  suav_def_temp_xc   = 1,
  proy_mor_temp_xc   = 'Best_proy',
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14,

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = 'met3_xm',
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det3_xm',
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,

  # Detalles del gráfico de calibración
  titulo             = 'Calibración Behavior Hipotercario Tradicional',
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.07,
  eje_y_xm           = 0.07,
  dim_grafico        = (22.5, 6),
  punt_mora          = 85,
  etiquetas          = True,
  tamaño_etiqueta    = 14,

  # Exportar factores para vista driver
  exportar_factores  = True
)

In [0]:
hm_bhv_cli_cal, rc, rm, fc_trad, fm_trad = Obj_Seg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "CODMES >= 202205 AND CAMPANIA_AGRUPADA_CLI = 'TRADICIONAL'",
  pd_seg             = pd_vig,
  pd_comparacion     = False,
  proys_def_temp     = 6,
  proys_mor_temp     = 3,
  rd_aux             = 'RD6',
  rd_aux2            = 'MORA_30_3',
  mora_temp          = 'MORA_30_3',

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = 'met3_xc',
  suav_def_temp_xc   = 1,
  proy_mor_temp_xc   = 'det3_xc',
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14,
  fact_to_proy_xc    = [fc_3_trad, fc_4_trad],


  # Configuración de proyecciones por montos
  proy_def_temp_xm   = 'met3_xm*0.95',
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det3_xm*0.9',
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,
  fact_to_proy_xm    = [fm_3_trad, fm_4_trad],

  # Detalles del gráfico de calibración
  titulo             = 'Calibración Behavior Hipotercario Tradicional',
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.05,
  eje_y_xm           = 0.05,
  dim_grafico        = (22.5, 6),
  punt_mora          = 85,
  etiquetas          = True,
  tamaño_etiqueta    = 14,

  # Exportar factores para vista driver
  exportar_factores  = True
)

In [0]:
driver='PERFIL_F1'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_Seg.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = "CODMES >= 202201 AND CAMPANIA_AGRUPADA_CLI='TRADICIONAL'",
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202201,
  codmes_fin1        = 202212,

  # Periodo reciente
  codmes_inicio2     = 202301,
  codmes_fin2        = 202412,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (22.5, 6),
  pos_leyenda        = (0.5, -0.25),
  etiquetas = True
)

In [0]:
Obj_SegDis, evoldis ,evolks= Obj_Seg.EvolDis(
  query_filtro = "CAMPANIA_AGRUPADA_CLI='TRADICIONAL'",
  pd_dis       = pd_vig,
  codmes_ini   = 202301,
  codmes_fin   = 202412,
  amplitud     = 1,
  disc_target= [12, 9,6],
  umbral_sup   = 0.5,
  umbral_inf   = 0.4,
  moviles      = False,
  etiquetas= True,
  tamaño_etiqueta=16,
  dim_grafico  = (22, 6),
  pos_leyenda  = (0.5, -0.3)
)

In [0]:
Obj_SegDis, evoldis ,evolks = Obj_Seg.EvolDis(
  query_filtro = "CODMES>=202101 AND CAMPANIA_AGRUPADA_CLI='TRADICIONAL' and PERFIL_F2='NO APALANCADO' ",
  pd_dis       = pd_vig,
  codmes_ini   = 202301,
  codmes_fin   = 202412,
  amplitud     = 1,
  disc_target= [12, 9,6],
  umbral_sup   = 0.5,
  umbral_inf   = 0.4,
  moviles      = False,
  eje_gini_max = 1,
  etiquetas= True,
  tamaño_etiqueta=20,
  dim_grafico  = (22, 6),
  pos_leyenda  = (0.5, -0.3)
)

In [0]:
Obj_SegDis, evoldis ,evolks = Obj_Seg.EvolDis(
  query_filtro = "CODMES>=202101 AND CAMPANIA_AGRUPADA_CLI='TRADICIONAL' and PERFIL_F2<>'NO APALANCADO' ",
  pd_dis       = pd_vig,
  codmes_ini   = 202301,
  codmes_fin   = 202412,
  amplitud     = 1,
  disc_target= [12, 9,6],
  umbral_sup   = 0.5,
  umbral_inf   = 0.4,
  moviles      = False,
  eje_gini_max = 1,
  etiquetas= True,
  tamaño_etiqueta=20,
  dim_grafico  = (22, 6),
  pos_leyenda  = (0.5, -0.3)
)

In [0]:
# Objeto de Seguimiento
Obj_Seg = rmm.MonitorScore_v01(
  nombre_modelo         = 'Personas Modelo Behavior',
  codigo_modelo         = 'MOD-BCP-20658',
  tier                  = 'II',
  detalle_seguimiento   = 'Seguimiento Behavior',
  mes_seguimiento       = '202502',
  base_monitoreo        = hip_f,
  pd1                   = pd_cal,
  pd2                   = pd_vig,
  monto_credito         = columna_monto,
  query_univ            = "CODMES >= 202201 AND DEF_60_BLOQ_REF_0=0 AND NUMEDADMADURACION_2>0 AND CAMPANIA_AGRUPADA_CLI = 'PYME'",
  bandas                = tipo_banda,
  codmes_default        = codmes_default,
  meses_ventana_target  = 12,
  meses_atraso_target   = 2
)

In [0]:
hm_bhv_cli_cal, rc, rm, fc_1_pyme, fm_1_pyme = Obj_Seg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "CODMES >= 202205 AND CAMPANIA_AGRUPADA_CLI = 'PYME' AND PERFIL_F1 = 'NO APALANCADO'",
  pd_seg             = pd_vig,
  pd_comparacion     = False,
  proys_def_temp     = 6,
  proys_mor_temp     = 3,
  rd_aux             = 'RD6',
  rd_aux2            = 'MORA_30_3',
  mora_temp          = 'MORA_30_3',

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = 'All_proy',
  suav_def_temp_xc   = 1,
  proy_mor_temp_xc   = 'All_proy',
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14,

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = 'met2_xm',
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det3_xm*0.95',
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,

  # Detalles del gráfico de calibración
  titulo             = 'Calibración Behavior Hipotercario Tradicional',
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.08,
  eje_y_xm           = 0.08,
  dim_grafico        = (22.5, 6),
  punt_mora          = 85,
  etiquetas          = True,
  tamaño_etiqueta    = 14,

  # Exportar factores para vista driver
  exportar_factores  = True
)

In [0]:
hm_bhv_cli_cal, rc, rm, fc_2_pyme, fm_2_pyme = Obj_Seg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "CODMES >= 202205 AND CAMPANIA_AGRUPADA_CLI = 'PYME' AND PERFIL_F1 <> 'NO APALANCADO'",
  pd_seg             = pd_vig,
  pd_comparacion     = False,
  proys_def_temp     = 6,
  proys_mor_temp     = 3,
  rd_aux             = 'RD6',
  rd_aux2            = 'MORA_30_3',
  mora_temp          = 'MORA_30_3',

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = 'All_proy',
  suav_def_temp_xc   = 1,
  proy_mor_temp_xc   = 'All_proy',
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14,

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = 'met2_xm',
  suav_def_temp_xm   = 0.5,
  proy_mor_temp_xm   = 'det3_xm',
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,

  # Detalles del gráfico de calibración
  titulo             = 'Calibración Behavior Hipotercario Tradicional',
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.08,
  eje_y_xm           = 0.08,
  dim_grafico        = (22.5, 6),
  punt_mora          = 85,
  etiquetas          = True,
  tamaño_etiqueta    = 14,

  # Exportar factores para vista driver
  exportar_factores  = True
)

In [0]:
hm_bhv_cli_cal, rc, rm, fc_2_pyme, fm_2_pyme = Obj_Seg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "CODMES >= 202205 AND CAMPANIA_AGRUPADA_CLI = 'PYME'",
  pd_seg             = pd_vig,
  pd_comparacion     = False,
  proys_def_temp     = 6,
  proys_mor_temp     = 3,
  rd_aux             = 'RD6',
  rd_aux2            = 'MORA_30_3',
  mora_temp          = 'MORA_30_3',

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = 'All_proy',
  suav_def_temp_xc   = 1,
  proy_mor_temp_xc   = 'All_proy',
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14,

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = 'met2_xm',
  suav_def_temp_xm   = 0.5,
  proy_mor_temp_xm   = 'det3_xm',
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,
  fact_to_proy_xm    = [fm_1_pyme, fm_2_pyme],

  # Detalles del gráfico de calibración
  titulo             = 'Calibración Behavior Hipotercario Tradicional',
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.08,
  eje_y_xm           = 0.08,
  dim_grafico        = (22.5, 6),
  punt_mora          = 85,
  etiquetas          = True,
  tamaño_etiqueta    = 14,

  # Exportar factores para vista driver
  exportar_factores  = True
)

In [0]:
Obj_SegDis, evoldis ,evolks= Obj_Seg.EvolDis(
  query_filtro = "CODMES>=202101",
  pd_dis       = pd_vig,
  codmes_ini   = 202301,
  codmes_fin   = 202412,
  amplitud     = 1,
  disc_target= [12, 9,6],
  umbral_sup   = 0.5,
  umbral_inf   = 0.4,
  moviles      = False,
  etiquetas= True,
  tamaño_etiqueta=16,
  dim_grafico  = (22, 6),
  pos_leyenda  = (0.5, -0.3)
)

In [0]:
Obj_SegDis, evoldis ,evolks = Obj_Seg.EvolDis(
  query_filtro = "CODMES>=202101 AND CAMPANIA_AGRUPADA_CLI='PYME' and PERFIL_F1<>'NO APALANCADO' ",
  pd_dis       = pd_vig,
  codmes_ini   = 202301,
  codmes_fin   = 202412,
  amplitud     = 1,
  disc_target= [12, 9,6],
  umbral_sup   = 0.5,
  umbral_inf   = 0.4,
  moviles      = False,
  eje_gini_max = 1,
  etiquetas= True,
  tamaño_etiqueta=20,
  dim_grafico  = (22, 6),
  pos_leyenda  = (0.5, -0.3)
)

In [0]:
Obj_SegDis, evoldis ,evolks = Obj_Seg.EvolDis(
  query_filtro = "CODMES>=202101 AND CAMPANIA_AGRUPADA_CLI='PYME' and PERFIL_F1='NO APALANCADO' ",
  pd_dis       = pd_vig,
  codmes_ini   = 202301,
  codmes_fin   = 202412,
  amplitud     = 1,
  disc_target= [12, 9,6],
  umbral_sup   = 0.5,
  umbral_inf   = 0.4,
  moviles      = False,
  eje_gini_max = 1,
  etiquetas= True,
  tamaño_etiqueta=20,
  dim_grafico  = (22, 6),
  pos_leyenda  = (0.5, -0.3)
)

In [0]:
base_bhv_gahi=spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_base_bhv_gahi_202503v2')

base_bhv_gahi = base_bhv_gahi.select('CODMES', 'CODCLAVECIC', 'CODCLAVEOPECTA', 'CAMPANIA_AGRUPADA_CLI', 'MTOPRINCIPAL_SOLES', 'PD_BHV_GAHI_3Q24', 'NUMEDADMADURACION_2', 'FLG_SEGMENT_RAN', 'SEGMENTO_BANCA_F', 'PD_BHV_FWL')

In [0]:
md_relcodclaveopectadl = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_MD_RELCODCLAVEOPECTADL')
md_relcodclaveopectadl = md_relcodclaveopectadl.withColumn("CODCLAVEOPECTA", col("codclaveopecta").cast("integer"))

# Tabla MIN_MES_DET: Obtener los minimos días de vencimiento 
hm_saldoprestamo = spark.table('catalog_lhcl_prod_bcp.bcp_udv_int_vu.h_saldocuentacreditopersonal')
hm_saldoprestamo = hm_saldoprestamo.withColumn("CODMES", date_format(col("FECDIA"), "yyyyMM"))
hm_saldoprestamo = hm_saldoprestamo.groupBy("CODMES", "CODCLAVECTA", "CTDDIAVCDA").agg(max("FECDIA").alias("MAX_FECDIA"))
min_mes_det = hm_saldoprestamo.groupBy("CODCLAVECTA").agg(
    min(when(col("CTDDIAVCDA") > 8, col("CODMES"))).alias("PRIMER_MES_8"),
    min(when(col("CTDDIAVCDA") > 15, col("CODMES"))).alias("PRIMER_MES_15"),
    min(when(col("CTDDIAVCDA") > 20, col("CODMES"))).alias("PRIMER_MES_20"),
    min(when(col("CTDDIAVCDA") > 30, col("CODMES"))).alias("PRIMER_MES_30")
)


# Cruzando para homologar el opecta
min_mes_det = min_mes_det.join(
    md_relcodclaveopectadl.selectExpr("CODCLAVECTADL as CODCLAVECTA", "CODCLAVEOPECTA"),
    on = ["CODCLAVECTA"],
    how = 'left'
)

min_mes_det.persist(StorageLevel.MEMORY_AND_DISK).count()

In [0]:
base_bhv_gahi = base_bhv_gahi.join(
    min_mes_det,
    on="codclaveopecta",
    how='left'
)

In [0]:
v_hm_univ_ctas_als = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.TABLASGE_v_hm_univ_ctas_als')
base_bhv_gahi = base_bhv_gahi.join(
    v_hm_univ_ctas_als.select("CODMES", "CODCLAVEOPECTA", "CTDDIAATRASO_F1", "CTDDIAATRASO_F2", "CTDDIAATRASO_F3"), 
    on=["CODCLAVEOPECTA", "CODMES"], 
    how='left'
)

In [0]:
# Tabla mm_mes_primer_default_norev: Obtener el minimo mes de default
mm_portafoliocredito = spark.table('catalog_lhcl_prod_bcp.bcp_ddv_adrmmgr_seginfobasesgenerales_vu.mm_portafoliocredito')

mm_mes_primer_default_norev = mm_portafoliocredito.join(
    md_relcodclaveopectadl.selectExpr("CODCLAVECTADL AS CODCLAVECTA", "CODCLAVEOPECTA"), 
    on = "CODCLAVECTA", 
    how="left"
)
base_bhv_gahi = base_bhv_gahi.drop('MIN_MES_DEF_60_BLOQ_REF_ACT')
base_bhv_gahi = base_bhv_gahi.join(
    mm_mes_primer_default_norev.select("codclaveopecta", coalesce(mm_mes_primer_default_norev["CODMESMINDEF60BLOQUEOREFIN"], lit(999999)).alias("MIN_MES_DEF_60_BLOQ_REF_ACT")),
    on="codclaveopecta",
    how='left'
)

In [0]:
base_bhv_gahi = base_bhv_gahi.withColumn(
    "PRIMER_MES_8", 
    when(col("PRIMER_MES_8").isNull(), 
    lit(999999)).otherwise(col("PRIMER_MES_8"))
)

base_bhv_gahi = base_bhv_gahi.withColumn(
    "DISTANCIA_8", 
    (F.round(col("PRIMER_MES_8")/100, 0)*12 + (col("PRIMER_MES_8")-F.round(col("PRIMER_MES_8")/100, 0)*100)) - 
    (F.round(col("CODMES")/100, 0)*12 + (col("CODMES")-F.round(col("CODMES")/100, 0)*100))
)

base_bhv_gahi = base_bhv_gahi.withColumn("DET_8_0", when(col("DISTANCIA_8") <= 0, lit(1)).otherwise(lit(0)))
base_bhv_gahi = base_bhv_gahi.withColumn("DET_8_2", when(col("DISTANCIA_8") <= 2, lit(1)).otherwise(lit(0)))
base_bhv_gahi = base_bhv_gahi.withColumn("DET_8_3", when(col("DISTANCIA_8") <= 3, lit(1)).otherwise(lit(0)))
base_bhv_gahi = base_bhv_gahi.withColumn("DET_8_4", when(col("DISTANCIA_8") <= 4, lit(1)).otherwise(lit(0)))

base_bhv_gahi = base_bhv_gahi.withColumn(
    "PRIMER_MES_15", 
    when(col("PRIMER_MES_15").isNull(), 
    lit(999999)).otherwise(col("PRIMER_MES_15"))
)

base_bhv_gahi = base_bhv_gahi.withColumn(
    "DISTANCIA_15", 
    (F.round(col("PRIMER_MES_15")/100, 0)*12 + (col("PRIMER_MES_15")-F.round(col("PRIMER_MES_15")/100, 0)*100)) - 
    (F.round(col("CODMES")/100, 0)*12 + (col("CODMES")-F.round(col("CODMES")/100, 0)*100))
)

base_bhv_gahi = base_bhv_gahi.withColumn("DET_15_0", when(col("DISTANCIA_15") <= 0, lit(1)).otherwise(lit(0)))
base_bhv_gahi = base_bhv_gahi.withColumn("DET_15_2", when(col("DISTANCIA_15") <= 2, lit(1)).otherwise(lit(0)))
base_bhv_gahi = base_bhv_gahi.withColumn("DET_15_3", when(col("DISTANCIA_15") <= 3, lit(1)).otherwise(lit(0)))
base_bhv_gahi = base_bhv_gahi.withColumn("DET_15_4", when(col("DISTANCIA_15") <= 4, lit(1)).otherwise(lit(0)))

base_bhv_gahi = base_bhv_gahi.withColumn(
    "PRIMER_MES_20", 
    when(col("PRIMER_MES_20").isNull(), 
    lit(999999)).otherwise(col("PRIMER_MES_20"))
)

base_bhv_gahi = base_bhv_gahi.withColumn(
    "DISTANCIA_20", 
    (F.round(col("PRIMER_MES_20")/100, 0)*12 + (col("PRIMER_MES_20")-F.round(col("PRIMER_MES_20")/100, 0)*100)) - 
    (F.round(col("CODMES")/100, 0)*12 + (col("CODMES")-F.round(col("CODMES")/100, 0)*100))
)


base_bhv_gahi = base_bhv_gahi.withColumn("DET_20_0", when(col("DISTANCIA_20") <= 0, lit(1)).otherwise(lit(0)))
base_bhv_gahi = base_bhv_gahi.withColumn("DET_20_2", when(col("DISTANCIA_20") <= 2, lit(1)).otherwise(lit(0)))
base_bhv_gahi = base_bhv_gahi.withColumn("DET_20_3", when(col("DISTANCIA_20") <= 3, lit(1)).otherwise(lit(0)))
base_bhv_gahi = base_bhv_gahi.withColumn("DET_20_4", when(col("DISTANCIA_20") <= 4, lit(1)).otherwise(lit(0)))

base_bhv_gahi = base_bhv_gahi.withColumn(
    "PRIMER_MES_30", 
    when(col("PRIMER_MES_30").isNull(), 
    lit(999999)).otherwise(col("PRIMER_MES_30"))
)

base_bhv_gahi = base_bhv_gahi.withColumn(
    "DISTANCIA_30", 
    (F.round(col("PRIMER_MES_30")/100, 0)*12 + (col("PRIMER_MES_30")-F.round(col("PRIMER_MES_30")/100, 0)*100)) - 
    (F.round(col("CODMES")/100, 0)*12 + (col("CODMES")-F.round(col("CODMES")/100, 0)*100))
)

base_bhv_gahi = base_bhv_gahi.withColumn("DET_30_0", when(col("DISTANCIA_30") <= 0, lit(1)).otherwise(lit(0)))
base_bhv_gahi = base_bhv_gahi.withColumn("DET_30_2", when(col("DISTANCIA_30") <= 2, lit(1)).otherwise(lit(0)))
base_bhv_gahi = base_bhv_gahi.withColumn("DET_30_3", when(col("DISTANCIA_30") <= 3, lit(1)).otherwise(lit(0)))
base_bhv_gahi = base_bhv_gahi.withColumn("DET_30_4", when(col("DISTANCIA_30") <= 4, lit(1)).otherwise(lit(0)))

base_bhv_gahi = base_bhv_gahi.withColumn('MORA_8_2',F.when(F.greatest(F.col('CTDDIAATRASO_F1'), F.col('CTDDIAATRASO_F2')) >= 8, 1).otherwise(0))
base_bhv_gahi = base_bhv_gahi.withColumn('MORA_15_2',F.when(F.greatest(F.col('CTDDIAATRASO_F1'), F.col('CTDDIAATRASO_F2')) >= 15, 1).otherwise(0))
base_bhv_gahi = base_bhv_gahi.withColumn('MORA_20_2',F.when(F.greatest(F.col('CTDDIAATRASO_F1'), F.col('CTDDIAATRASO_F2')) >= 20, 1).otherwise(0))
base_bhv_gahi = base_bhv_gahi.withColumn('MORA_30_2',F.when(F.greatest(F.col('CTDDIAATRASO_F1'), F.col('CTDDIAATRASO_F2')) >= 30, 1).otherwise(0))
base_bhv_gahi = base_bhv_gahi.withColumn('MORA_45_2',F.when(F.greatest(F.col('CTDDIAATRASO_F1'), F.col('CTDDIAATRASO_F2')) >= 45, 1).otherwise(0))

base_bhv_gahi = base_bhv_gahi.withColumn('MORA_1_3',F.when(F.greatest(F.col('CTDDIAATRASO_F1'), F.col('CTDDIAATRASO_F2'), F.col('CTDDIAATRASO_F3'))>= 1, 1).otherwise(0))
base_bhv_gahi = base_bhv_gahi.withColumn('MORA_8_3',F.when(F.greatest(F.col('CTDDIAATRASO_F1'), F.col('CTDDIAATRASO_F2'), F.col('CTDDIAATRASO_F3'))>= 8, 1).otherwise(0))
base_bhv_gahi = base_bhv_gahi.withColumn('MORA_15_3',F.when(F.greatest(F.col('CTDDIAATRASO_F1'), F.col('CTDDIAATRASO_F2'), F.col('CTDDIAATRASO_F3'))>= 15, 1).otherwise(0))
base_bhv_gahi = base_bhv_gahi.withColumn('MORA_20_3',F.when(F.greatest(F.col('CTDDIAATRASO_F1'), F.col('CTDDIAATRASO_F2'), F.col('CTDDIAATRASO_F3'))>= 20, 1).otherwise(0))
base_bhv_gahi = base_bhv_gahi.withColumn('MORA_30_3',F.when(F.greatest(F.col('CTDDIAATRASO_F1'), F.col('CTDDIAATRASO_F2'), F.col('CTDDIAATRASO_F2'))>= 30, 1).otherwise(0))
base_bhv_gahi = base_bhv_gahi.withColumn('MORA_45_3',F.when(F.greatest(F.col('CTDDIAATRASO_F1'), F.col('CTDDIAATRASO_F2'), F.col('CTDDIAATRASO_F2'))>= 45, 1).otherwise(0))

base_bhv_gahi.persist(StorageLevel.MEMORY_AND_DISK).count()

base_bhv_gahi = base_bhv_gahi.withColumn(
    "MIN_MES_DEF", 
    when(col("MIN_MES_DEF_60_BLOQ_REF_ACT").isNull(), 
    lit(999999)).otherwise(col("MIN_MES_DEF_60_BLOQ_REF_ACT"))
)

base_bhv_gahi = base_bhv_gahi.withColumn(
    "DISTANCIA_DEF", 
    (F.round(col("MIN_MES_DEF")/100, 0)*12 + (col("MIN_MES_DEF")-F.round(col("MIN_MES_DEF")/100, 0)*100)) - 
    (F.round(col("CODMES")/100, 0)*12 + (col("CODMES")-F.round(col("CODMES")/100, 0)*100))
)

base_bhv_gahi = base_bhv_gahi.withColumn("DEF12", when(col("DISTANCIA_DEF") <= 12, lit(1)).otherwise(lit(0)))
base_bhv_gahi = base_bhv_gahi.withColumn("DEF11", when(col("DISTANCIA_DEF") <= 11, lit(1)).otherwise(lit(0)))
base_bhv_gahi = base_bhv_gahi.withColumn("DEF10", when(col("DISTANCIA_DEF") <= 10, lit(1)).otherwise(lit(0)))
base_bhv_gahi = base_bhv_gahi.withColumn("DEF9", when(col("DISTANCIA_DEF") <= 9, lit(1)).otherwise(lit(0)))
base_bhv_gahi = base_bhv_gahi.withColumn("DEF8", when(col("DISTANCIA_DEF") <= 8, lit(1)).otherwise(lit(0)))
base_bhv_gahi = base_bhv_gahi.withColumn("DEF7", when(col("DISTANCIA_DEF") <= 7, lit(1)).otherwise(lit(0)))
base_bhv_gahi = base_bhv_gahi.withColumn("DEF6", when(col("DISTANCIA_DEF") <= 6, lit(1)).otherwise(lit(0)))
base_bhv_gahi = base_bhv_gahi.withColumn("DEF5", when(col("DISTANCIA_DEF") <= 5, lit(1)).otherwise(lit(0)))
base_bhv_gahi = base_bhv_gahi.withColumn("DEF4", when(col("DISTANCIA_DEF") <= 4, lit(1)).otherwise(lit(0)))
base_bhv_gahi = base_bhv_gahi.withColumn("DEF3", when(col("DISTANCIA_DEF") <= 3, lit(1)).otherwise(lit(0)))
base_bhv_gahi = base_bhv_gahi.withColumn("DEF2", when(col("DISTANCIA_DEF") <= 2, lit(1)).otherwise(lit(0)))
base_bhv_gahi = base_bhv_gahi.withColumn("DEF1", when(col("DISTANCIA_DEF") <= 1, lit(1)).otherwise(lit(0)))

# Def foto
base_bhv_gahi = base_bhv_gahi.withColumn("DEF_60_BLOQ_REF_0", when(col("DISTANCIA_DEF") <= 0, lit(1)).otherwise(lit(0)))

base_bhv_gahi.persist(StorageLevel.MEMORY_AND_DISK).count()

In [0]:
base_bhv_gahi.createOrReplaceTempView("temp_view")
# Usar el parámetro en la consulta SQL
spark.sql(f"""
    CREATE OR REPLACE TABLE catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_base_bhv_gahi_202505_2
    LOCATION 'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/HIPOTECARIO_2025/BHV/202505_2/base_bhv_gahi'
    AS SELECT distinct a.* FROM temp_view as a
""")

In [0]:
gahi = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_base_bhv_gahi_202505_2')

In [0]:
pd_vig='PD_BHV_CLI_GHI'
pd_cal='PD_BHV_GAHI_3Q24'# en uso
pd_trc='xb_nue_bhv'
tipo_banda = 'Vasiseck'#Jeffrey Vasiseck
codmes_default = 202406
columna_monto='MTOPRINCIPAL_SOLES'
titulo='Calibración Behavior Garantia Hipotecaria'
condiciones= "CODMES >= 202208 AND NUMEDADMADURACION_2>0"

In [0]:
# Objeto de Seguimiento
Obj_Seg = rmm.MonitorScore_v01(
  nombre_modelo         = 'Personas Modelo Behavior',
  codigo_modelo         = 'MOD-BCP-20658',
  tier                  = 'II',
  detalle_seguimiento   = 'Seguimiento Behavior',
  mes_seguimiento       = '202502',
  base_monitoreo        = gahi,
  pd1                   = pd_cal,
  pd2                   = pd_cal,
  monto_credito         = columna_monto,
  query_univ            = "CODMES >= 202201 AND DEF_60_BLOQ_REF_0=0 AND NUMEDADMADURACION_2>0",
  bandas                = tipo_banda,
  codmes_default        = codmes_default,
  meses_ventana_target  = 12,
  meses_atraso_target   = 2
)

In [0]:
Obj_Seg_cal, rc1, rm1, fc1, fm1 = Obj_Seg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "CODMES>=202201 and PD_BHV_GAHI_3Q24 IS NOT NULL AND FLG_SEGMENT_RAN='1. Ingresos altos al día' AND SEGMENTO_BANCA_F='1. Afluente'",
  pd_seg             = pd_cal,
  pd_comparacion     = True,
  proys_def_temp     = 6,
  proys_mor_temp     = 3,
  rd_aux             = 'RD6',
  rd_aux2            = 'MORA_30_3', # Corregido
  mora_temp          = 'MORA_30_3',

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = 'All_proy', #All_proy met1_xc met2_xc met3_xc
  suav_def_temp_xc   = 1,
  proy_mor_temp_xc   = 'All_proy', #All_proy det1_xc det2_xc det3_xc
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14,

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = "np.where(calxmontos['CODMES'].isin([202410, 202411, 202412]), met1_xm*1.6, met1_xm*1.1)", #All_proy met1_xm met2_xm met3_xm
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det3_xm*1.2', #All_proy det1_xm det2_xm det3_xm
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,
  #fact_to_proy_xc    = [fc_1, fc_2],
  #fact_to_proy_xm    = [fm_1, fm_2],

  # Detalles del gráfico de calibración
  titulo             = titulo,
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.04,
  eje_y_xm           = 0.04,
  dim_grafico        = (25, 6.5),
  punt_mora          = 85, #Tamaño de puntos de mora temprana
  etiquetas          =True,
  pos_etiquetas_xm   =10,
  pos_etiquetas_xc   =10,
  tamaño_etiqueta    =20,

  # Exportar factores para vista driver
  exportar_factores  = True
)

In [0]:
Obj_Seg_cal, rc2, rm2, fc2, fm2 = Obj_Seg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "CODMES>=202201 and PD_BHV_GAHI_3Q24 IS NOT NULL AND FLG_SEGMENT_RAN <>'1. Ingresos altos al día' AND SEGMENTO_BANCA_F='1. Afluente'",
  pd_seg             = pd_cal,
  pd_comparacion     = True,
  proys_def_temp     = 6,
  proys_mor_temp     = 3,
  rd_aux             = 'RD6',
  rd_aux2            = 'MORA_30_3', # Corregido
  mora_temp          = 'MORA_30_3',

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = 'met3_xc', #All_proy met1_xc met2_xc met3_xc
  suav_def_temp_xc   = 1,
  proy_mor_temp_xc   = 'det3_xc', #All_proy det1_xc det2_xc det3_xc
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14,

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = "(met1_xm + met2_xm)/2", #All_proy met1_xm met2_xm met3_xm
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det3_xm', #All_proy det1_xm det2_xm det3_xm
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,
  #fact_to_proy_xc    = [fc_1, fc_2],
  #fact_to_proy_xm    = [fm_1, fm_2],

  # Detalles del gráfico de calibración
  titulo             = titulo,
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.3,
  eje_y_xm           = 0.3,
  dim_grafico        = (25, 6.5),
  punt_mora          = 85, #Tamaño de puntos de mora temprana
  etiquetas          =True,
  pos_etiquetas_xm   =10,
  pos_etiquetas_xc   =10,
  tamaño_etiqueta    =20,

  # Exportar factores para vista driver
  exportar_factores  = True
)

In [0]:
Obj_Seg_cal, rc3, rm3, fc3, fm3 = Obj_Seg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "CODMES>=202201 and PD_BHV_GAHI_3Q24 IS NOT NULL and  SEGMENTO_BANCA_F<>'1. Afluente'",
  pd_seg             = pd_cal,
  pd_comparacion     = True,
  proys_def_temp     = 6,
  proys_mor_temp     = 3,
  rd_aux             = 'RD6',
  rd_aux2            = 'MORA_30_3', # Corregido
  mora_temp          = 'MORA_30_3',

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = 'met3_xc', #All_proy met1_xc met2_xc met3_xc
  suav_def_temp_xc   = 1,
  proy_mor_temp_xc   = 'det3_xc', #All_proy det1_xc det2_xc det3_xc
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14,

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = "(met1_xm + met2_xm)/2", #All_proy met1_xm met2_xm met3_xm
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det3_xm', #All_proy det1_xm det2_xm det3_xm
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,
  #fact_to_proy_xc    = [fc_1, fc_2],
  #fact_to_proy_xm    = [fm_1, fm_2],

  # Detalles del gráfico de calibración
  titulo             = titulo,
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.15,
  eje_y_xm           = 0.15,
  dim_grafico        = (25, 6.5),
  punt_mora          = 85, #Tamaño de puntos de mora temprana
  etiquetas          =True,
  pos_etiquetas_xm   =10,
  pos_etiquetas_xc   =10,
  tamaño_etiqueta    =20,

  # Exportar factores para vista driver
  exportar_factores  = True
)

In [0]:
Obj_Seg_cal, rc3, rm3, fc3, fm3 = Obj_Seg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "CODMES>=202201 and PD_BHV_GAHI_3Q24 IS NOT NULL and  SEGMENTO_BANCA_F<>'1. Afluente'",
  pd_seg             = pd_cal,
  pd_comparacion     = True,
  proys_def_temp     = 6,
  proys_mor_temp     = 3,
  rd_aux             = 'RD6',
  rd_aux2            = 'MORA_30_3', # Corregido
  mora_temp          = 'MORA_30_3',

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = 'met3_xc', #All_proy met1_xc met2_xc met3_xc
  suav_def_temp_xc   = 1,
  proy_mor_temp_xc   = 'det3_xc', #All_proy det1_xc det2_xc det3_xc
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14,

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = "(met1_xm + met2_xm)/2", #All_proy met1_xm met2_xm met3_xm
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det3_xm', #All_proy det1_xm det2_xm det3_xm
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,
  #fact_to_proy_xc    = [fc_1, fc_2],
  #fact_to_proy_xm    = [fm_1, fm_2],

  # Detalles del gráfico de calibración
  titulo             = titulo,
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.15,
  eje_y_xm           = 0.15,
  dim_grafico        = (25, 6.5),
  punt_mora          = 85, #Tamaño de puntos de mora temprana
  etiquetas          =True,
  pos_etiquetas_xm   =10,
  pos_etiquetas_xc   =10,
  tamaño_etiqueta    =20,

  # Exportar factores para vista driver
  exportar_factores  = True
)

In [0]:
Obj_Seg_cal, rcG, rmG, fcG, fmG = Obj_Seg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "CODMES>=202201 and PD_BHV_GAHI_3Q24 IS NOT NULL ",
  pd_seg             = pd_cal,
  pd_comparacion     = True,
  proys_def_temp     = 6,
  proys_mor_temp     = 3,
  rd_aux             = 'RD6',
  rd_aux2            = 'MORA_30_3', # Corregido
  mora_temp          = 'MORA_30_3',

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = 'met3_xc', #All_proy met1_xc met2_xc met3_xc
  suav_def_temp_xc   = 1,
  proy_mor_temp_xc   = 'det3_xc', #All_proy det1_xc det2_xc det3_xc
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14,

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = "(met1_xm + met2_xm)/2", #All_proy met1_xm met2_xm met3_xm
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det3_xm', #All_proy det1_xm det2_xm det3_xm
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,
  fact_to_proy_xm    = [fm1, fm2,fm3],

  # Detalles del gráfico de calibración
  titulo             = titulo,
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.07,
  eje_y_xm           = 0.07,
  dim_grafico        = (25, 6.5),
  punt_mora          = 85, #Tamaño de puntos de mora temprana
  etiquetas          =True,
  pos_etiquetas_xm   =10,
  pos_etiquetas_xc   =10,
  tamaño_etiqueta    =20,

  # Exportar factores para vista driver
  exportar_factores  = True
)

In [0]:
rmG

In [0]:
Obj_SegDis, evoldis ,evolks = Obj_Seg.EvolDis(
  query_filtro = "CODMES>=202101 AND FLG_SEGMENT_RAN ='1. Ingresos altos al día' AND SEGMENTO_BANCA_F='1. Afluente'",
  pd_dis       = pd_cal,
  codmes_ini   = 202301,
  codmes_fin   = 202412,
  amplitud     = 1,
  disc_target= [12, 9,6],
  umbral_sup   = 0.5,
  umbral_inf   = 0.4,
  moviles      = False,
  etiquetas= True,
  tamaño_etiqueta=20,
  dim_grafico  = (35, 7),
  pos_leyenda  = (0.5, -0.3)
)

In [0]:
Obj_SegDis, evoldis ,evolks = Obj_Seg.EvolDis(
  query_filtro = "CODMES>=202101 AND ((FLG_SEGMENT_RAN <>'1. Ingresos altos al día' AND SEGMENTO_BANCA_F='1. Afluente') or SEGMENTO_BANCA_F<>'1. Afluente')",
  pd_dis       = pd_cal,
  codmes_ini   = 202301,
  codmes_fin   = 202412,
  amplitud     = 1,
  disc_target= [12, 9,6],
  umbral_sup   = 0.5,
  umbral_inf   = 0.4,
  moviles      = False,
  etiquetas= True,
  tamaño_etiqueta=20,
  dim_grafico  = (35, 7),
  pos_leyenda  = (0.5, -0.3)
)

In [0]:
Obj_SegDis, evoldis ,evolks = Obj_Seg.EvolDis(
  query_filtro = "CODMES>=202101 ",
  pd_dis       = pd_cal,
  codmes_ini   = 202301,
  codmes_fin   = 202412,
  amplitud     = 1,
  disc_target= [12, 9, 6],
  umbral_sup   = 0.5,
  umbral_inf   = 0.4,
  moviles      = False,
  etiquetas= True,
  tamaño_etiqueta=20,
  dim_grafico  = (22, 6),
  pos_leyenda  = (0.5, -0.3)
)